I need to install qiime on the cluster, then I go:

In [1]:
!cat /etc/*-release 

CentOS Linux release 7.2.1511 (Core) 
NAME="CentOS Linux"
VERSION="7 (Core)"
ID="centos"
ID_LIKE="rhel fedora"
VERSION_ID="7"
PRETTY_NAME="CentOS Linux 7 (Core)"
ANSI_COLOR="0;31"
CPE_NAME="cpe:/o:centos:centos:7"
HOME_URL="https://www.centos.org/"
BUG_REPORT_URL="https://bugs.centos.org/"

CENTOS_MANTISBT_PROJECT="CentOS-7"
CENTOS_MANTISBT_PROJECT_VERSION="7"
REDHAT_SUPPORT_PRODUCT="centos"
REDHAT_SUPPORT_PRODUCT_VERSION="7"

CentOS Linux release 7.2.1511 (Core) 
CentOS Linux release 7.2.1511 (Core) 


To install QIIME, I am following instructions from [here](https://gist.github.com/alanorth/100dd95f809223ae0100)

In [5]:
!module load Python
#!module load R

In [7]:
!pip install --user qiime -I

  Using cached qiime-1.9.1.tar.gz
  Using cached scipy-0.17.0-cp27-cp27m-manylinux1_x86_64.whl
  Using cached cogent-1.5.3.tgz
  Using cached natsort-3.5.6-py2.py3-none-any.whl
  Using cached pynast-1.2.2.tar.gz
  Using cached qcli-0.1.1.tar.gz
  Using cached gdata-2.0.18.tar.gz
    100% |################################| 317kB 1.7MB/s 
  Using cached emperor-0.9.51.tar.gz
  Using cached scikit-bio-0.2.3.tar.gz
    100% |################################| 215kB 1.7MB/s 
  Using cached burrito-0.9.1.tar.gz
  Using cached qiime-default-reference-0.1.3.tar.gz
  Using cached click-6.6.tar.gz
  Using cached future-0.15.2.tar.gz
  Using cached pyqi-0.3.2.tar.gz
  Running setup.py install for cogent ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Running setup.py install for pynast ... - \ done
  Running setup.py install for qcli ... - \ done
  Running setup.py install for

In [1]:
!print_qiime_config.py -t


System information
         Platform:	linux2
   Python version:	2.7.9 (default, Apr 10 2015, 15:04:25)  [GCC 4.8.2 20140120 (Red Hat 4.8.2-16)]
Python executable:	/software/Python/2.7.9/bin/python

QIIME default reference information
For details on what files are used as QIIME's default references, see here:
 https://github.com/biocore/qiime-default-reference/releases/tag/0.1.3

Dependency versions
          QIIME library version:	1.9.1
           QIIME script version:	1.9.1
qiime-default-reference version:	0.1.3
                  NumPy version:	1.10.4
                  SciPy version:	0.17.0
                 pandas version:	0.18.0
             matplotlib version:	1.4.3
            biom-format version:	2.1.5
                   h5py version:	Not installed.
                   qcli version:	0.1.1
                   pyqi version:	0.3.2
             scikit-bio version:	0.2.3
                 PyNAST version:	1.2.2
                Emperor version:	0.9.51
                burrito version:	0.9.1

Goal: To match IDs to OTUs - this will enable me to select OTU differentially enriched based on the LEfSE analysis,  
and return transcripts, based on the reference correlation performed here: /Volumes/group_dv/personal/DValenzano/papers/uBiome0/data/regr.csv  
First, I take the OTU genus reference table, normalise it and save it as a csv

In [2]:
#!export PATH=/beegfs/group_dv/home/DValenzano/.Python/2.7/lib/python2.7/site-packages/qiime/:$PATH
#!export PATH=/export/apps/qiime/1.8.0/bin:$PATH
#!export PATH=$PATH:/software/R/3.2.4/bin/
#!export PATH=$PATH:/software/R/3.2.4/bin
#!echo $PATH
#ls ~/.bashrc

To install all the required R packages, follow instructions from http://qiime.org/install/install.html

In [1]:
#!module load R
#!which R
get_ipython().system(u'normalize_table.py -i /beegfs/group_dv/home/DValenzano/uBiome/data/Description_otu_table_sorted_L6.biom -a CSS -o /beegfs/group_dv/home/DValenzano/uBiome/data/Description_otu_table_sorted_L6n.biom')

This file I just normalized has the OTUs **at the genus level** for all the individuals for which we also have the rnaseq data 

In [2]:
get_ipython().system(u'biom convert -i /beegfs/group_dv/home/DValenzano/uBiome/data/Description_otu_table_sorted_L6n.biom -o /beegfs/group_dv/home/DValenzano/uBiome/data/Description_otu_table_sorted_L6n.txt --to-tsv')

In [ ]:
get_ipython().system(u'biom convert -i /beegfs/group_dv/home/DValenzano/uBiome/data/Description_otu_table_sorted_L6n.biom -o /beegfs/group_dv/home/DValenzano/uBiome/data/Description_otu_table_sorted_L6n.txt --to-tsv')

In [45]:
get_ipython().system(u'biom convert -i /beegfs/group_dv/home/DValenzano/uBiome/data/16wk_otu_table.biom -o /beegfs/group_dv/home/DValenzano/uBiome/data/16wk_otu_table.txt --to-tsv')

In [10]:
t = open('/beegfs/group_dv/home/DValenzano/uBiome/data/Description_otu_table_sorted_L6n.txt', 'rU').read()
t = t.replace('\t',',')
z = open('/beegfs/group_dv/home/DValenzano/uBiome/data/Description_otu_table_sorted_L6n.csv', 'w')
z.write(t)
z.close()

Now I need to generate a new correlation between fpkm and OTU, individual by individual  
The OTU Table is [this](/beegfs/group_dv/home/DValenzano/uBiome/data/Description_otu_table_sorted_L6n.csv), which now needs to be transposed and sorted  
I am following the same guidelines as in [here](https://github.com/dvalenzano/Interactive-Sessions/blob/master/11-Mar-2016.py)  

In [11]:
t1 = t.replace('_','.').replace('YI', 'WT')
t2 = t1.split('\n')[1:]
t3 = [ i.split(',') for i in t2 ]
t4 = [t3[0]]+[i for i in t3[1:] if sum(map(float, (i[1:]))) != 0.0] #this removes 0-only OTUs
tt = zip(*t4)
tT = [ list(i) for i in tt ]

In [12]:
u = open('/beegfs/group_dv/home/DValenzano/uBiome/data/rnaseq_fpkm_genes.csv', 'rU').read()
u2 = u.split('\n')
u3 = [ i.split(',') for i in u2]
u4 = [u3[0]]+[i for i in u3[1:] if sum(map(float, (i[1:]))) != 0.0] #this removes 0-only transcripts
ut = zip(*u4)
uT = [ list(i) for i in ut ]

In [13]:
from sets import Set
uh = [i[0] for i in uT[1:]]
th = [i[0] for i in tT[1:]]
h = list(Set(uh) & Set(th))
tT1 = [tT[0]] + [i for i in tT if i[0] in h]
uT1 = [uT[0]] + [i for i in uT if i[0] in h]

/beegfs/group_dv/home/DValenzano/.Python/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: the sets module is deprecated
  if __name__ == '__main__':


In [14]:
uTs = [uT1[0]] + sorted(uT1[1:], key = lambda x: (x[0]))
tTs = [tT1[0]] + sorted(tT1[1:], key = lambda x: (x[0]))

#[i[0] for i in uTs][1:] == [i[0] for i in tTs][1:] # this tests whether the two matrices have headers in same order

uj = ','.join([','.join(i)+'\n' for i in uTs]).replace('\n,','\n')[:-1]
tj = ','.join([','.join(i)+'\n' for i in tTs]).replace('\n,','\n')[:-1]

In [63]:
zu = open('/beegfs/group_dv/home/DValenzano/uBiome/data/rnaseq_fpkm_genes_sorted.csv', 'w')
zu.write(uj)
zu.close()

zt = open('/beegfs/group_dv/home/DValenzano/uBiome/data/rnaseq_merged_otu_table_sorted.csv', 'w')
zt.write(tj)
zt.close()

In [67]:
[i.split(',')[0] for i in tj.split('\n')[1:-1]] == [i.split(',')[0] for i in uj.split('\n')[1:-1]]

True

There is a difference from the previous analysis (March 17-18 on README) since 3 samples were removed due to low depth  
Good - Now continuing from [here](https://github.com/dvalenzano/Interactive-Sessions/blob/master/15-Mar-2016.py)

In [7]:
import numpy as np
from scipy.stats.stats import pearsonr
from scipy import stats
import pandas as pd
from scipy import stats, integrate
import matplotlib.pyplot as plt
#import seaborn as sns

In [16]:
rna = uj
otu = tj

In [36]:
rnal = [i.split(',') for i in rna.split('\n')] #that's a "l" as in "L"
otul = [i.split(',') for i in otu.split('\n')]

rnat = [ list(i) for i in zip(*rnal)]
otut = [list(i) for i in zip(*otul) ]

rnat_z = [ map(float, i[1:]) for i in rnat[1:] if i.count('0') < 5]
otut_z = [ map(float, i[1:]) for i in otut[1:] if i.count('0.0') < 5]


# In[10]:

rna_key = [i[0] for i in rnat[1:] if i.count('0') < 5]
otu_key = [ i[0] for i in otut[1:] if i.count('0.0') < 5]

rna_value = rnat_z
otu_value = otut_z

rna_d = dict(zip(rna_key, rna_value))
otu_d = dict(zip(otu_key, otu_value))

In [ ]:
def loop_one(m1,otu): # this returns all the correlation coefficients between m1 and a given otu (otu)
    ls = []
    for i in m1:
        ls.append(stats.linregress(i, otu))    #this function returns slope, intercept, r_value, p_value, std_err
    return ','.join(map(str, ls))+'\n'

def loop_two(m1, m2):
    ls = []
    for j in m2:
        ls.append(loop_one(m1, j))
    return ','.join(ls).replace('\n,','\n')

fpkmvsotu = loop_two(rnat_z, otut_z)
fpkmvsotu1 = fpkmvsotu.replace('LinregressResult', '').replace('slope=','').replace('intercept=', '').replace('rvalue=', '').replace('pvalue=', '').replace('stderr=', '') 

z = open('/beegfs/group_dv/home/DValenzano/uBiome/data/corr.csv','w')
z.write(fpkmvsotu1)
z.close()

In [2]:
class reg(object):
    
    """
    This class uses as input a linear regression file obtained by the loop_two function and returns 
    regression tables and plots, given specific parameteres, such as R-squared value, p-value, OTU or 
    transcript
    """
            
#     version = 0.1
    
#     def __init__(self, inp):
#         self.inp = inp 
#         self.inp2 = [ i.split('LinregressResult')[1:] for i in self.inp.split('\n')[:-1]] 
#         def loop1(inp, n):
#             ls = []
#             for i in inp:
#                 ls.append(i.split(',')[n].split('=')[1])
#             return ','.join(ls).replace(')', '')
#         def loop2(inp, n):
#             ls = []
#             for i in inp:
#                 ls.append(loop1(i, n))
#             return ls
#         self.slope =  [map(float, i.split(',')) for i in loop2(self.inp2, 0)]
#         self.intercept = [map(float, i.split(',')) for i in loop2(self.inp2, 1)]
#         self.rvalue = [map(float, i.split(',')) for i in loop2(self.inp2, 2)]
#         self.rsquare = [ map(lambda x: x**2, i) for i in self.rvalue]
#         self.pvalue = [map(float, i.split(',')) for i in loop2(self.inp2, 3)]
#         self.stderr0 = [map(float, i.split(',')) for i in loop2(self.inp2, 4)  ]  

    version = 0.2
    
    def __init__(self, inp):
        self.inp = inp 
        self.inp2 = [ i.split('(') for i in self.inp.split('\n')[:-1]] 
    
        def loop1(inp, n):
            ls = []
            for i in inp[1:]:
                ls.append(i.split(',')[n])
            return ','.join(ls).replace(')', '')
        def loop2(inp, n):
            ls = []
            for i in inp:
                ls.append(loop1(i, n))
            return ls
       
        self.slope =  [map(float, i.split(',')) for i in loop2(self.inp2, 0)]
        self.intercept = [map(float, i.split(',')) for i in loop2(self.inp2, 1)]
        self.rvalue = [map(float, i.split(',')) for i in loop2(self.inp2, 2)]
        self.rsquare = [ map(lambda x: x**2, i) for i in self.rvalue]
        self.pvalue = [map(float, i.split(',')) for i in loop2(self.inp2, 3)]
        self.stderr = [map(float, i.split(',')) for i in loop2(self.inp2, 4)  ]   


In [3]:
def filt(inp, thr, sign): #filter p-values, r-values and so forth based on a chosen threshold
    ar = np.array(inp)
    ls = []
    if sign == '>':
        ls = zip(*np.where(ar > thr))
    elif sign == '<':
        ls = zip(*np.where(ar < thr))
    else: 
        print "Error: check your input file"
    return ls


def filt_(inp, thr, sign): #filter p-values, r-values and so forth based on a chosen threshold
    ar = np.array(inp)
    li = []
    lp = []
    if sign == '>':
        lp = np.where(ar > thr)
    elif sign == '<':
        lp = np.where(ar < thr)
    else: 
        print "Error: check your input file"
    p1_l = [ list(i) for i in lp]
    la = list(ar[p1_l])
    p2 = p1_l+[la]
    p3 = [ list(i) for i in zip(*p2)]
    p4 = sorted(p3, key=lambda x: x[2])
    return ','.join([ ','.join(map(str, i))+'\n' for i in p4]).replace('\n,','\n')[:-1]

In [5]:
#fpkmvsotu = open('/beegfs/group_dv/home/DValenzano/uBiome/data/corr.csv', 'rU').read()
#fpkmvsotu1 = fpkmvsotu.replace('LinregressResult', '').replace('slope=','').replace('intercept=', '').replace('rvalue=', '').replace('pvalue=', '').replace('stderr=', '') 

rf = reg(fpkmvsotu1)

In [18]:
rpval = filt_(rf.pvalue, 0.001, '<')
P2 = [otu_key[int(i.split(',')[0])]+','+rna_key[int(i.split(',')[1])] +','+i.split(',')[2] for i in rpval.split('\n') ]
P3 = ','.join([ i+'\n' for i in P2]).replace('\n,','\n')[:-1]
len(P3.split('\n'))

zpvalue = 'OTU,transcr,value\n'+','.join([i+'\n'for i in P3.split('\n')[:100]]).replace('\n,','\n')
z = open('/beegfs/group_dv/home/DValenzano/uBiome/data/TOP_otu-tr_pvalue.csv', 'w')
z.write(zpvalue)
z.close()

Below the instructions to plot 

In [ ]:
Rf = reg(fpkmvsotu1)
Rpval = filt_(Rf.pvalue, 0.00001, '<') #all the pvals < 0.00001
Rrsq = filt_(Rf.rsquare, 0.8, '>') #all the rsquare > 0.8